# Match tensor
## NOTE: no where close to fully functional.

In [143]:
import json
import os
import sys
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing import sequence
from keras.layers import merge, Dense, Input,Dropout, Embedding, LSTM, Bidirectional, Activation
from keras.layers import Conv2D
from keras.layers.merge import dot, multiply, add
from keras.layers import Merge
from keras.layers.core import Lambda,Reshape
from keras.layers.pooling import GlobalMaxPooling2D
from keras.models import Model
from keras.backend import transpose
from HomeDepotCSVReader import HomeDepotReader
import Utilities
from DataPreprocessing import DataPreprocessing
from Feature_Word2Vec import Feature_Word2Vec
from AutomaticQueryExpansion import Word2VecQueryExpansion

In [2]:
train_filename = '../data/train_play.csv'
test_filename = '../data/test_play.csv'
attribute_filename = '../data/attributes_play.csv'
description_filename = '../data/product_descriptions_play.csv'
word2vec_model_path='model/word2vec_play.model'
vocab_path='model/word2vec_play_vocab.json'
embeddings_path='model/embeddings_play.npz'

# train_filename = '../data/train.csv'
# test_filename = '../data/test.csv'
# attribute_filename = '../data/attributes.csv'
# description_filename = '../data/product_descriptions.csv'
# word2vec_model_path='model/word2vec.model'
# vocab_path='model/word2vec_vocab.json'
# embeddings_path='model/embeddings.npz'

In [3]:
reader = HomeDepotReader()

train_query_df, product_df, attribute_df, test_query_df = reader.getQueryProductAttributeDataFrame(train_filename,
                                              test_filename,
                                              attribute_filename,
                                              description_filename)
print("train_query_df:",list(train_query_df))
print("product_df:", list(product_df))
print("attribute_df:", list(attribute_df))
print("test_query_df:", list(test_query_df))

===========Tranforming labels...
showing current values
trainDF: ['id', 'product_uid', 'search_term', 'relevance']
trainDF:    id  product_uid    search_term  relevance
0   2       100001  angle bracket        3.0
self.mergedLabelDF: ['relevance'] 
 <class 'pandas.core.frame.DataFrame'> (11, 1)    relevance
0        3.0
Old unique Labels: [ 2.33  2.5   2.67  3.  ]
newLabels: [0 1 2 3]
Creating new column for training:  relevance_int
===========Transform labels completed
train_query_df: ['id', 'product_uid', 'search_term', 'relevance', 'relevance_int', 'product_idx']
product_df: ['product_title', 'product_uid', 'product_description']
attribute_df: ['product_uid', 'name', 'value']
test_query_df: ['id', 'product_uid', 'search_term']


In [4]:
#transform attribute into doc
dp = DataPreprocessing()
attribute_doc_df = dp.getAttributeDoc(attribute_df)
#attribute_doc_df
product_df=product_df.join(attribute_doc_df.set_index('product_uid'), on = 'product_uid')

In [5]:
product_df['content'] = train_query_df['search_term'].map(str) + " " + \
                        product_df['product_title'].map(str) + " " + \
                        product_df['product_description'].map(str) + " " + \
                        product_df['attr_json'].map(str)
        
# ## no attribute
# product_df['content'] = product_df['product_title'].map(str) + " " + \
#                         product_df['product_description'].map(str) 
        
product_df['content']

0    angle bracket Simpson Strong-Tie 12-Gauge Angl...
1    l bracket BEHR Premium Textured DeckOver 1-gal...
2    deck over Delta Vero 1-Handle Shower Only Fauc...
3    rain shower head Whirlpool 1.9 cu. ft. Over th...
4    shower only faucet Lithonia Lighting Quantum 2...
5    convection otr House of Fara 3/4 in. x 3 in. x...
6    microwave over stove Valley View Industries Me...
Name: content, dtype: object

In [6]:

w2v=Feature_Word2Vec(modelFilename=word2vec_model_path)
sentences=w2v.convertDFIntoSentences(product_df,'content')
print(sentences)
w2v.trainModel(sentences)

Loading Word2Vec model
[['angle', 'bracket', 'Simpson', 'Strong-Tie', '12-Gauge', 'Angle', 'Not', 'only', 'do', 'angles', 'make', 'joints', 'stronger,', 'they', 'also', 'provide', 'more', 'consistent,', 'straight', 'corners'], ['Simpson', 'Strong-Tie', 'offers', 'a', 'wide', 'variety', 'of', 'angles', 'in', 'various', 'sizes', 'and', 'thicknesses', 'to', 'handle', 'light-duty', 'jobs', 'or', 'projects', 'where', 'a', 'structural', 'connection', 'is', 'needed'], ['Some', 'can', 'be', 'bent', '(skewed)', 'to', 'match', 'the', 'project'], ['For', 'outdoor', 'projects', 'or', 'those', 'where', 'moisture', 'is', 'present,', 'use', 'our', 'ZMAX', 'zinc-coated', 'connectors,', 'which', 'provide', 'extra', 'resistance', 'against', 'corrosion', '(look', 'for', 'a', '"Z"', 'at', 'the', 'end', 'of', 'the', 'model', 'number)'], ['Versatile', 'connector', 'for', 'various', '90', 'connections', 'and', 'home', 'repair', 'projectsStronger', 'than', 'angled', 'nailing', 'or', 'screw', 'fastening', 'alo

In [7]:
w2v.model.wv.vocab

{"'Assembled": <gensim.models.keyedvectors.Vocab at 0x7f5b79454b38>,
 "'Bullet01':": <gensim.models.keyedvectors.Vocab at 0x7f5b79454ba8>,
 "'Bullet02':": <gensim.models.keyedvectors.Vocab at 0x7f5b79454630>,
 "'Bullet03':": <gensim.models.keyedvectors.Vocab at 0x7f5b79454668>,
 "'Bullet04':": <gensim.models.keyedvectors.Vocab at 0x7f5b794546a0>,
 "'Bullet05':": <gensim.models.keyedvectors.Vocab at 0x7f5b794546d8>,
 "'MFG": <gensim.models.keyedvectors.Vocab at 0x7f5b79454710>,
 "'No',": <gensim.models.keyedvectors.Vocab at 0x7f5b79454cc0>,
 "'Number": <gensim.models.keyedvectors.Vocab at 0x7f5b794547b8>,
 "'Product": <gensim.models.keyedvectors.Vocab at 0x7f5b794547f0>,
 "'Yes',": <gensim.models.keyedvectors.Vocab at 0x7f5b79454c18>,
 '(in': <gensim.models.keyedvectors.Vocab at 0x7f5b79454860>,
 ")':": <gensim.models.keyedvectors.Vocab at 0x7f5b79454898>,
 '3': <gensim.models.keyedvectors.Vocab at 0x7f5b79454550>,
 '9': <gensim.models.keyedvectors.Vocab at 0x7f5b79454dd8>,
 'BEHR': <ge

In [8]:
w2vExpand=Word2VecQueryExpansion(modelFilename=word2vec_model_path)
query="cooking"
print("Expanding query: ")
print(w2vExpand.getExpandedQuery(query,maxNoOfAdditionalWords=2,minSimilarityLevel=0.65,))

Expanding query: 
Loading Word2Vec model
 cooking


In [9]:
print(w2v.getVectorFromWord('wood'))
print(w2v.getSimilarWordVectors('wood',5))
print(len(w2v.getVectorFromWord('wood')))

[  5.41748479e-03   2.85501708e-03  -2.32536742e-03   3.81654385e-03
  -5.53219579e-03   4.81992261e-03  -1.71320245e-03  -4.32056870e-04
   5.07620350e-03   2.30746693e-03  -3.32578900e-03  -5.37868077e-03
  -4.37597558e-03  -3.07537848e-03   5.89979289e-04   5.18816058e-03
   1.27569947e-03  -4.78184735e-03   1.07518479e-03  -3.17750615e-03
   3.96114076e-03   5.20368107e-03   4.35851980e-03  -5.95842139e-05
   9.96039133e-04  -9.07552079e-04  -2.17679935e-03   4.56288131e-03
  -5.00325160e-03  -1.98955205e-03   1.28543680e-03   2.35484052e-03
   4.29834286e-03  -2.53780192e-04  -2.26583797e-03  -1.39929564e-03
  -3.99282231e-04   1.23241718e-03   2.32636376e-04  -7.93315587e-04
   3.55244172e-03   1.62934652e-03   1.88487791e-03   4.02611418e-04
  -6.59624988e-04  -4.34085540e-03   1.72889128e-03   3.54781095e-03
   7.43209559e-04  -2.58179102e-03   2.15714425e-03  -4.63416893e-03
  -2.86066136e-03  -1.75713294e-03   1.03036780e-03  -2.07389891e-03
  -1.19056075e-03   1.38393091e-03

# embeddings to keras
http://ben.bolte.cc/resources/embeddings/embeddings.html

In [10]:
vocab = dict([(k, v.index) for k, v in w2v.model.wv.vocab.items()])
with open(vocab_path, 'w') as f:
    f.write(json.dumps(vocab))

In [11]:
weights = w2v.model.wv.syn0
np.save(open(embeddings_path, 'wb'), weights)

In [12]:
def load_vocab(vocab_path):
    """
    Load word -> index and index -> word mappings
    :param vocab_path: where the word-index map is saved
    :return: word2idx, idx2word
    """

    with open(vocab_path, 'r') as f:
        data = json.loads(f.read())
    word2idx = data
    idx2word = dict([(v, k) for k, v in data.items()])
    return word2idx, idx2word

In [13]:
word2idx, idx2word = load_vocab(vocab_path)

In [14]:
def word2vec_embedding_layer(embeddings_path):
    """
    Generate an embedding layer word2vec embeddings
    :param embeddings_path: where the embeddings are saved (as a numpy file)
    :return: the generated embedding layer
    """

    weights = np.load(open(embeddings_path, 'rb'))
    layer = Embedding(input_dim=weights.shape[0], output_dim=weights.shape[1], weights=[weights])
    return layer

In [15]:
print_tokens=True

In [16]:
# variable arguments are passed to gensim's word2vec model
# if options.train:
#     print('Training Word2Vec...')
#     create_embeddings(options.data, options.embeddings, options.vocab, size=100, min_count=5, window=5, sg=1, iter=25)

word2idx, idx2word = load_vocab(vocab_path)

if print_tokens:
    print('Tokens:', ', '.join(word2idx.keys()))



Tokens: a, variety, of, in, and, to, or, is, can, be, the, use, for, at, are, 3, x, from, with, 'Bullet02':, 'Bullet03':, 'Bullet04':, 'Bullet05':, 'MFG, Brand, Name':, 'Number, 'Product, Depth, (in, )':, Height, Width, BEHR, an, It, your, wood, concrete, The, look, ft, 'Assembled, in',, 'Bullet01':, Family':, 'Yes',, Resistant':, Type':, 'No',, when, on, not, that, 9, Sensor, this, microwave, cooking, have, you, time, interior, all, power, fast, fan, light, stakes


# convert to idx

In [95]:
def query_sent2idx(df,col):
    query_word2vec_idx_list=[]
    queries=w2v.convertDFIntoSentences(df,col)
    print(len(queries))
    for query in queries:
        idx_list = []
        for word in query:
            if word not in word2idx.keys():
                idx_list+=[0]#[len(word2idx.keys())] # use last as special key #TODO: well we need to fix this. Using 0 for now so it's in range To OOV or something random
            else:
                idx_list+=[word2idx[word]]
        query_word2vec_idx_list+=[idx_list]
        print("=====")
        print(idx_list)
        print("=====")
    return query_word2vec_idx_list


In [96]:
query_word2vec_idx_list = query_sent2idx(train_query_df,'search_term')
print(query_word2vec_idx_list)

11
=====
[0, 0]
=====
=====
[0, 0]
=====
=====
[0, 0]
=====
=====
[0, 0, 0]
=====
=====
[0, 0, 0]
=====
=====
[0, 0]
=====
=====
[30, 0, 0]
=====
=====
[0]
=====
=====
[0, 68]
=====
=====
[0, 0]
=====
=====
[0, 0]
=====
[[0, 0], [0, 0], [0, 0], [0, 0, 0], [0, 0, 0], [0, 0], [30, 0, 0], [0], [0, 68], [0, 0], [0, 0]]


In [97]:
# def doc_sent2idx(df,col):
#     doc_word2vec_idx_list=[]
#     for row in df.iteritems():
#         a=w2v.convertDFIntoSentences(row,col)
#         print(a)
# #    print(len(queries))
# #     for query in queries:
# #         idx_list = []
# #         for word in query:
# #             if word not in word2idx.keys():
# #                 idx_list+=[len(word2idx.keys())] # use last as special key
# #             else:
# #                 idx_list+=[word2idx[word]]
# #         query_word2vec_idx_list+=[idx_list]
# #         print("=====")
# #         print(idx_list)
# #         print("=====")
#     return doc_word2vec_idx_list

In [98]:
#TODO: this is fucked. just bodge for testing nn
doc_word2vec_idx_list = query_sent2idx(product_df,'product_title')
print(doc_word2vec_idx_list)

14
=====
[0, 0, 0, 0]
=====
=====
[55, 0, 0, 0, 0]
=====
=====
[0, 0, 0, 0, 0, 0]
=====
=====
[0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0]
=====
=====
[0, 0]
=====
=====
[65, 0]
=====
=====
[20]
=====
=====
[0, 1, 0, 0, 0, 5, 0, 0, 9, 47, 0]
=====
=====
[0, 0, 0, 0, 0, 0, 0, 0, 0]
=====
=====
[0, 3, 0, 0, 5]
=====
=====
[19, 37, 5]
=====
=====
[19, 0, 20]
=====
=====
[0, 0, 0]
=====
=====
[0, 0, 0, 0, 0, 0]
=====
[[0, 0, 0, 0], [55, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0], [0, 0], [65, 0], [20], [0, 1, 0, 0, 0, 5, 0, 0, 9, 47, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 3, 0, 0, 5], [19, 37, 5], [19, 0, 20], [0, 0, 0], [0, 0, 0, 0, 0, 0]]


In [99]:
# # cosine similarity model
# print('Building model...')
# input_a = Input(shape=(1,), dtype='int32', name='input_a')
# input_b = Input(shape=(1,), dtype='int32', name='input_b')
# embeddings = word2vec_embedding_layer(embeddings_path)
# embedding_a = embeddings(input_a)
# embedding_b = embeddings(input_b)
# similarity = merge([embedding_a, embedding_b], mode='cos', dot_axes=2)
# model = Model(input=[input_a, input_b], output=similarity)
# model.compile(optimizer='sgd', loss='mse') # optimizer and loss don't matter


# word_a = 'wood'#raw_input('First word: ')
# if word_a not in word2idx:
#     print('"%s" is not in the index' % word_a)
# word_b = 'fan'#raw_input('Second word: ')
# if word_b not in word2idx:
#     print('"%s" is not in the index' % word_b)
# output = model.predict([np.asarray([word2idx[word_a]]), np.asarray([word2idx[word_b]])])
# print('%f' % output)

# Define model

In [100]:
query_max_len = 6 #covers 95.74% of the search lengths (see data exploration)
doc_max_len = 400 # todo: confirm this is sensible

In [145]:
print('Building model...')

def get_R(x):
    a, b = x.values()
    return K.dot(a, b)


embeddings = word2vec_embedding_layer(embeddings_path)

#embedding lookup
# TODO: replace this with embedding with mask zero rather than padding, need to change index for encoding too, imput dim too.
# TODO: OOV embedding
input_query = Input(shape=(query_max_len,), dtype='int32', name='input_query')
input_doc = Input(shape=(doc_max_len,), dtype='int32', name='input_doc')
embedding_query = embeddings(input_query)  # (None, 6, 100)
embedding_doc = embeddings(input_doc)      # (None, 400, 100)

#shared linear projection
shared_lp = Dense(40,activation='linear')
query_output = shared_lp(embedding_query) # (None, 6, 40) 
doc_output = shared_lp(embedding_doc) #(None, 400, 40) 

#query: bi LSTM, lp -- implementation = 0 for CPU option, 1 or 2 for GPU
query_output = Bidirectional(LSTM(30, dropout=0.5, implementation=0, return_sequences=True, go_backwards=True)\
                             ,merge_mode='mul'#TODO: alts are 'sum','mul','ave','concat'<--default, None
                            )(query_output) #(None, 6, 30) unless concat (None, 6, 60)  
query_output = Dense(50,activation='linear')(query_output) #(None, 50)

#doc: bi LSTM, lp
doc_output = Bidirectional(LSTM(30, dropout=0.5, implementation=0, return_sequences=True, go_backwards=True)\
                             ,merge_mode='mul'#TODO: alts are 'sum','mul','ave','concat'<--default, None
                            )(doc_output) #(None, 6, 400) unless concat (None, 6, 800)  
doc_output = Dense(50,activation='linear')(doc_output) #(None, 50)

#2d product
#mt_input = dot([query_output, doc_output], axes=(2), normalize=False)  
#output is (11, 6, 400), where did 50 go?
######################################

def state_layer_dot_prod(x):
    print(x[0][0])
    for i in range(50):
        #output = dot([x[0], x[1]], axes=(2), normalize=False)  
        output = dot([x[0][i], transpose(x[1][i])], axes=(0,1), normalize=False)  
    
    return output

def state_layer_dot_prod_shape(input_shape):
    'Merge output shape'
    shape = list(input_shape)
    #print(input_shape)
    #print(shape)
    outshape = (shape[0][0],shape[1][1],shape[0][1],shape[0][2])
    #print(outshape)
    return tuple(outshape)

######################################
mt_input = Lambda(state_layer_dot_prod,output_shape=state_layer_dot_prod_shape)([query_output, doc_output])

##TODO fix error:
#InvalidArgumentError: Input to reshape is a tensor with 4800 values, but the requested shape requires a multiple of 120000
#[[Node: reshape_23/Reshape = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](lambda_40/dot_42/MatMul, reshape_23/Reshape/shape)]]
#mt_input = Reshape((doc_max_len, query_max_len,50))(mt_input)

# #Append exact match channel
# #TODO:

# # Conv layers 1
# output1 = Conv2D(filters=18, kernel_size=(3, 3), strides=(1,1), padding='valid', data_format='channels_last', \
#                 dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
#                 bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
#                 activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(mt_input)

# output2 = Conv2D(filters=18, kernel_size=(3, 4), strides=(1,1), padding='valid', data_format='channels_last', \
#                 dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
#                 bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
#                 activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(mt_input)

# output3 = Conv2D(filters=18, kernel_size=(3, 5), strides=(1,1), padding='valid', data_format='channels_last', \
#                 dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
#                 bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
#                 activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(mt_input)

# # Conv layers 2
# output1 = Conv2D(filters=20, kernel_size=(1, 1), strides=(1,1), padding='valid', data_format='channels_last', \
#                 dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
#                 bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
#                 activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(output1)

# output2 = Conv2D(filters=20, kernel_size=(1, 1), strides=(1,1), padding='valid', data_format='channels_last', \
#                 dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
#                 bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
#                 activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(output2)

# output3 = Conv2D(filters=20, kernel_size=(1, 1), strides=(1,1), padding='valid', data_format='channels_last', \
#                 dilation_rate=(1, 1), activation='relu', use_bias=True, kernel_initializer='glorot_uniform', \
#                 bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, \
#                 activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(output3)

# # Max pool layer
# output1 = GlobalMaxPooling2D(data_format='channels_last')(output1)
# output2 = GlobalMaxPooling2D(data_format='channels_last')(output2)
# output3 = GlobalMaxPooling2D(data_format='channels_last')(output3)

# # Merge
# output_combined = add([output1,output2,output3])

# # Final layer
# output = Dense(1,activation='sigmoid')(output_combined)


# build model
#model = Model([input_query], [query_output])
#model = Model([input_doc], [encoded_doc])
#model = Model([input_query,input_doc], [output1,output2,output3])
#model = Model([input_query,input_doc], [output_combined])
#model = Model([input_query,input_doc], [query_output])
#model = Model([input_query,input_doc], [query_output,doc_output])
#model = Model([input_query,input_doc], [output])
model = Model([input_query,input_doc], [mt_input])


model.summary()

Building model...
Tensor("lambda_51/strided_slice:0", shape=(6, 50), dtype=float32)


IndexError: tuple index out of range

In [ ]:
# from keras.layers import Input, merge
# from keras.models import Model
# import numpy as np

# input_a = np.reshape([[1, 2, 3],[1, 2, 3],],[[1, 2, 3],[1, 2, 3]])
# input_b = np.reshape([4, 5, 6], (1, 1, 3))

# print(input_a)
# print(input_b)
# a = Input(shape=(1, 3))
# b = Input(shape=(1, 3))

# concat = merge([a, b], mode='concat', concat_axis=-1)
# dot = merge([a, b], mode='dot', dot_axes=2)
# cos = merge([a, b], mode='cos', dot_axes=2)

# model_concat = Model(input=[a, b], output=concat)
# model_dot = Model(input=[a, b], output=dot)
# model_cos = Model(input=[a, b], output=cos)

# print(model_concat.predict([input_a, input_b]))
# print(model_dot.predict([input_a, input_b]))
# print(model_cos.predict([input_a, input_b]))

In [138]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [139]:
x_query_train=np.array(query_word2vec_idx_list)
x_doc_train=np.array(doc_word2vec_idx_list)[0:len(x_query_train)]

# TODO: replace this with embedding with mask zero rather than padding, need to change index for encoding too, imput dim too.
print("Pad sequences (samples x time)")
x_query_train = sequence.pad_sequences(x_query_train, maxlen=query_max_len)
x_doc_train = sequence.pad_sequences(x_doc_train, maxlen=doc_max_len)
# x_query_test = sequence.pad_sequences(x_query_test, maxlen=query_max_len)
# x_doc_test = sequence.pad_sequences(x_doc_test, maxlen=doc_max_len)

print('x_query_train shape:', x_query_train.shape)
print('x_doc_train shape:', x_doc_train.shape)
# print('x_query_test shape:', x_query_test.shape)
# print('x_doc_test shape:', x_doc_test.shape)

#TODO: y label on 0-1 scale
y_train=train_query_df['relevance_int'].as_matrix()
# y_test=test_query_df['relevance_int'].as_matrix()


Pad sequences (samples x time)
x_query_train shape: (11, 6)
x_doc_train shape: (11, 400)


In [140]:
batch_size=2
print('Train...')
hist=model.fit([x_query_train,x_doc_train], [y_train],
          batch_size=batch_size,
          epochs=3,
          #validation_data=[x_test, y_test])
         )


Train...


ValueError: Error when checking model target: expected lambda_47 to have 4 dimensions, but got array with shape (11, 1)

In [ ]:
batch_size=2
prediction=model.predict([x_query_train,x_doc_train],
          batch_size=batch_size,
         )

In [ ]:
prediction.shape

In [ ]:
print("Predict")
word_a = 'wood'#raw_input('First word: ')
if word_a not in word2idx:
    print('"%s" is not in the index' % word_a)
word_b = 'fan'#raw_input('Second word: ')
if word_b not in word2idx:
    print('"%s" is not in the index' % word_b)
output = model.predict([np.asarray([word2idx[word_a]]), np.asarray([word2idx[word_b]])])
print('%f' % output)